In [2]:
pip install transformers[torch] datasets torch scikit-learn

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------- ----------------------------- 2.6/10.5 MB 13.7 MB/s eta 0:00:01
   --------------------- ------------------ 5.5/10.5 MB 14.0 MB/s eta 0:00:01
   -------------------------------- ------- 8.4/10.5 MB 14.1 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 13.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ------------------ --------------------- 2.9/6.2 MB 15.2 MB/s eta 0:00:01
   ------------------------------------- -- 5.8/6.2 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.1 MB 13.7 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 13.6 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.1 MB 13.7 MB/s eta 0:00:01
   ----------

In [1]:
import os
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import datetime


c:\Users\Admin\anaconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "distilgpt2"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"../model/fine-tuned-{timestamp}"


In [3]:
# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # required
model = GPT2LMHeadModel.from_pretrained(model_name)

In [4]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(device)

cuda


In [5]:
# Load dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="../data/cleaned_emails_final.txt",
    block_size=128,
)

c:\Users\Admin\anaconda3\envs\deeplearning\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [7]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=500,
    prediction_loss_only=True,
    report_to="none",
)


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [9]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,4.277700
1000,3.933300
1500,3.785400
2000,3.687900
2500,3.655700
3000,3.458800
3500,3.427700
4000,3.425400
4500,3.367500
5000,3.390300


TrainOutput(global_step=5060, training_loss=3.636182421657879, metrics={'train_runtime': 3370.2877, 'train_samples_per_second': 6.005, 'train_steps_per_second': 1.501, 'total_flos': 661080778997760.0, 'train_loss': 3.636182421657879, 'epoch': 2.0})

In [10]:
# Save model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved at {output_dir}")

✅ Model saved at ../model/fine-tuned-20250603_134959


In [11]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
def autocomplete_gpt2(prompt, max_new_tokens=5):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,         # makes it feel "smart"
        top_k=30,
        top_p=0.9,
        temperature=0.5,
        repetition_penalty=1.1
    )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return full_text[len(prompt):].rsplit('.', 1)[0].strip()  # return only new suggestion


In [20]:
autocomplete_gpt2("Continue this email in a helpful tone and stop after end of sentence or '.': wanna grad dinner next")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


"week? i'm going"

In [14]:
texts = [
    'here is',
    'have a',
    'please review',
    'please call me',
    'thanks for',
    'let me',
    'Let me know',
    'Let me know if you',
    'this sounds',
    'is this call going to',
    'can you get',
    'is it okay',
    'it should',
    'call if there\'s',
    'gave her a',
    'i will let',
    'i will be',
    'may i get a copy of all the',
    'how is our trade',
    'this looks like a',
    'i am fine with the changes',
    'please be sure this'
]
import pandas as pd
output = list(map(autocomplete_gpt2, texts))
output_df = pd.DataFrame({'input': texts, 'output': output})
# output_df.head(len(output))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

In [63]:
output_df

,input,output
0,here is,a very good place to
1,have a,chance to win
2,please review,and act upon this request
3,please call me,at and I will be
4,thanks for,the opportunity to meet with
5,let me,know what you think
6,Let me know,if you have any questions
7,Let me know if you,have any questions
8,this sounds,like a good idea
9,is this call going to,be a good one
